In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

%aimport decoder
%aimport globalmaptiles
%aimport utils

In [ ]:
ZOOM = 15
# TILESET = "mapbox.mapbox-terrain-v2"
# TILESET = "mapbox.mapbox-streets-v7"
TILESET = "trailsroc.ckaix6nvo0lgb2ak7gxrf6lpk-0cnr1"
LAYER = "prod-v1-build1"
SEG_FILTER = "mponds"
EXTENT = 4096

In [ ]:
trails = {}
TX, TY = utils.get_mponds_tiles(ZOOM)
for x, tx in enumerate(TX):
    for y, ty in enumerate(TY):
        print(x, y, tx, ty, ZOOM)
        try:
            tile = utils.get_tile(TILESET, tx, ty, ZOOM)
        except Exception as e:
            print(f"Error getting tile {tx} {ty} {ZOOM}: {e}")
            tile = {
                "prod-v1-build1": {
                    "extent": 4096,
                    "version": 2,
                    "features": []
                }
            }
        
        layer = tile[LAYER]
        assert layer["extent"] == EXTENT, f"expected layer extent == {EXTENT}"
        tile_trails = [
            utils.TrailSegment(f) for f in layer["features"]
            if f["properties"].get("trailsroc-type") == "trailSegment" and
            SEG_FILTER in f["properties"].get("trailsroc-id")
        ]
#         tile_trails = [trail.clip(EXTENT, EXTENT) for trail in tile_trails]
#         tile_trails = [trail for trail in tile_trails if len(trail.coords) > 0]
        tile_trails = [trail.shift(x*EXTENT, y*EXTENT) for trail in tile_trails]
        trails[(x, y)] = tile_trails

In [ ]:
tids = set(trail.id for k in trails for trail in trails[k])
for tid in tids:
    tiles = [k for k in trails for trail in trails[k] if trail.id == tid]
    parts = [trail for k in trails for trail in trails[k] if trail.id == tid]
    if len(tiles) == 1:
        continue
    elif len(tiles) == 2:
        t1, t2 = tiles
        (t1x, t1y), (t2x, t2y) = t1, t2
        dx, dy = abs(t1x-t2x), abs(t1y-t2y)
        assert (dx==0 and dy==1) or (dx==1 and dy==0)
    elif len(tiles) == 3:
        print(tiles, tid)
    elif len(tiles) == 4:
        print(tiles, tid)
    else:
        raise Exception("too many parts")

In [ ]:
for tile, trail in zip(tiles, parts):
    for coords in trail.coords:
        print(tile, coords)
        plt.plot(*np.array(coords).T, marker="o")
ax = plt.axis()
for x in set(x for x, y in tiles):
    plt.axvline(x*EXTENT)
for y in set(y for x, y in tiles):
    plt.axhline(y*EXTENT)
plt.axis(ax)

In [ ]:
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
for i, (x, y) in enumerate(trails):
    color = "C" + str(i)
    if len(trails[(x, y)]) == 0:
        plt.plot([x*EXTENT, (x+1)*EXTENT], [y*EXTENT, (y+1)*EXTENT], color=color)
        plt.plot([x*EXTENT, (x+1)*EXTENT], [(y+1)*EXTENT, y*EXTENT], color=color)
    else:
        for trail in trails[(x, y)]:
            if len(trail.coords) > 1:
                assert len(trail.coords) < 4, len(trail.coords)
                plt.plot(*np.array(trail.coords[0]).T, color="red", lw=5)
                plt.plot(*np.array(trail.coords[1]).T, color="green", lw=5)
                if len(trail.coords) > 2:
                    plt.plot(*np.array(trail.coords[2]).T, color="blue", lw=5)
            for coords in trail.coords:
                plt.plot(*np.array(coords).T, color=color)

    plt.plot([x*EXTENT, x*EXTENT, (x+1)*EXTENT, (x+1)*EXTENT, x*EXTENT],
             [y*EXTENT, (y+1)*EXTENT, (y+1)*EXTENT, y*EXTENT, y*EXTENT],
             color=color)

In [ ]:
for i, (x, y) in enumerate(trails):
    for trail in trails[(x, y)]:
        if len(trail.coords) > 1:
            print(i, x, y, trail.id, len(trail.coords))
            print(trail.coords)
            print()

In [ ]:
xx = []
for i, (x, y) in enumerate(features):
    for feature in features[(x, y)]:
        if feature["properties"]["trailsroc-type"] == "trailSegment":
            xx.append(utils.TrailSegment(feature))
        type_ = feature["geometry"]["type"]
        tr_type = feature["properties"]["trailsroc-type"]
        tr_id = feature["properties"]["trailsroc-id"]
        tr_name = feature["properties"].get("trailsroc-name")
        if tr_type == "trailSegment":
            assert "trailsroc-color" in feature["properties"]
        type_tup = (type_, tr_type)
        print("%15s %20s %s %s" % (type_tup + (tr_id, tr_name)))

In [ ]:
xx = set()
for i, (x, y) in enumerate(features):
    for feature in features[(x, y)]:
        if feature["properties"]["trailsroc-type"] == "trailSegment":
            xx.add(feature["properties"].get("trailsroc-name", "Unknown"))

In [ ]:
xx

In [ ]:
# L = tiles[(0, 1)]
# R = tiles[(1, 1)]
# trail = "seg:trail-mponds-unnamed-trails:ee21931c"
L = tiles[(1, 1)]
R = tiles[(1, 0)]
trail = "seg:trail-mponds-quaker:1440c1cd"
trail = "seg:trail-mponds-smeadow:a9101492"
trail = "seg:trail-mponds-unnamed-trails:8ec2ef08"
trail = "seg:trail-mponds-unnamed-trails:62914ce7"
trail = "seg:trail-mponds-unnamed-trails:f9984151"


Lf, = [f for f in L["features"] if f["properties"]["trailsroc-id"] == trail]
Rf, = [f for f in R["features"] if f["properties"]["trailsroc-id"] == trail]

Lc = np.array(Lf["geometry"]["coordinates"])
Rc = np.array(Rf["geometry"]["coordinates"])

In [ ]:
if len(Lc) > len(Rc):
    plt.plot(*Lc.T, marker="o", lw=3)
    plt.plot(*Rc.T, marker="o")
else:
    plt.plot(*Rc.T, marker="o", lw=3)
    plt.plot(*Lc.T, marker="o")

In [ ]:
df = []
for x, y in features:
    for feature in features[(x, y)]:
        df.append(feature["properties"])
df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
def plot_features(features, xs=0, ys=0, color=None):
    for feature in features:
        geometry = feature["geometry"]
        type_ = geometry["type"]
        coords = geometry["coordinates"]
        if type_ == "Point":
            x, y = coords
            plt.plot(x+xs, y+ys, marker=".", color=color)
        elif type_ == "MultiPoint":
            for point in coords:
                x, y = point
                plt.plot(x+xs, y+ys, marker="x", color=color)
        elif type_ == "Polygon" or type_ == "MultiPolygon":
            for polygon in coords:
                x, y = np.array(polygon).T
                plt.plot(x+xs, y+ys, linestyle="--", color=color)
        elif type_ == "LineString":
            x, y = np.array(coords).T
            plt.plot(x+xs, y+ys, color=color)
        elif type_ == "MultiLineString":
            for line_string in coords:
                x, y = np.array(line_string).T
                plt.plot(x+xs, y+ys, color=color)
        else:
            raise Exception("Unknown type: " + type_)

In [ ]:
# graph stuff

In [ ]:
G = nx.read_weighted_edgelist('mendonponds_edgelist.txt')

In [ ]:
nx.shortest_path(G, 'BL', 'BL')

In [ ]:
chk_pts = ["BL", "W11", "HP1", "DB", "Y9", "G15", "C4", "P8", "R16", "G3"]
route = nx.approximation.traveling_salesman_problem(G, nodes=chk_pts)
sum(G[a][b]['weight'] for a, b in nx.utils.pairwise(route))

In [ ]:
# first task: input list of nodes, output adjacency matrix of shortest path lengths between pairs
nodes = ['BL', 'HP1', 'G15', 'R17']
adj_m = np.zeros((len(nodes), len(nodes)))
for i, j in itertools.combinations(range(len(nodes)), r=2):
    n1, n2 = nodes[i], nodes[j]
    path = nx.shortest_path(G, n1, n2)
    adj_m[i, j] = sum(G[a][b]['weight'] for a, b in nx.utils.pairwise(path))

H = nx.from_numpy_array(adj_m)

In [ ]:
adj_m = np.array([[0, 1, 2, 3],
                  [0, 0, 4, 5],
                  [0, 0, 0, 6],
                  [0, 0, 0, 0]])
H = nx.from_numpy_array(adj_m)

In [ ]:
nx.approximation.traveling_salesman_problem(G, nodes=)

In [ ]:
def bf_tsp(H):
    print(H)
    
bf_tsp(H)

In [ ]:
for p in itertools.permutations(range(len(H))):
    print(p)

In [ ]:
nx.approximation.traveling_salesman_problem(H, nodes=[1, 2, 3])

In [ ]:
def draw_graph_with_labels(G):
    pos = nx.spring_layout(G)
    labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx(G, pos)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)